# <center>**Insurance Premium Prediction**</center>

# **Introduction**

# **Problem Statement**

## **Importing required Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

import logging
import pickle
import joblib

ModuleNotFoundError: No module named 'pandas'

In [2]:
# Setting up logging
logging.basicConfig(filename='app.log', filemode='w', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

NameError: name 'logging' is not defined

## **Importing dataset**

In [ ]:
Insurance = pd.read_csv("/content/insurance.csv")

: 

In [ ]:
print(Insurance.shape)
Insurance.head()

: 

**Obsrvation:**
The dataset contain 7 columns and 1338 rows.
7 columns are age, sex, bmi,childern, smoker, region, expenses.

# **Data Description**

In [ ]:
logger.info('Data Description')

: 

**The data present the different columns have the meaning as follows:**

- age: age of primary beneficiary

- sex: insurance contractor gender, female, male

- bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

- children: Number of children covered by health insurance / Number of dependents

- smoker: Smoking

- region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

- charges: Individual medical costs billed by health insurance

In [ ]:
# Information of the Insurance dataset
Insurance.info()

: 

**Observation:**
- Here we can see the total number of columns, columns name, columns data type and memory usage of the dataset.
- In this dataset we have 2 float64, 2 int64, and 3 object
- In this dataset the expenses column is the target column and remaining are the independent columns. 


# **Data Exploration**

In [ ]:
logger.info('Data Exploration')

: 

In [ ]:
# checking the duplicated values
Insurance.duplicated().sum()

: 

**Observation:**
we have 1 duplicated value so we can drop that.

In [ ]:
Insurance = Insurance.drop_duplicates()

: 

**Observation:**
Now we have 1337 rows with 7 columns.

In [ ]:
# checking null values
Insurance.isnull().sum()

: 

**Observation:**
The dataset doesn't contain null values.


In [ ]:
Insurance.describe()

: 

**Observation:**
* The Average Age of the person is 40, and the minimum age is 18 and maximum age is 64.
* 

In [ ]:
# Identify the dataset columns into numerical and categorical columns
categorical_columns = []
numerical_columns = []

for i in Insurance:
  if Insurance[i].dtype=='object':
    categorical_columns.append(i)
  else:
    numerical_columns.append(i)

: 

In [ ]:
numerical_columns

: 

In [ ]:
categorical_columns

: 

In [ ]:
# checking the unquie values in categorical columns
for i in categorical_columns:
  print('*'*50)
  print('value count for:',i)
  print(Insurance[i].value_counts())
  print('*'*50)

: 

# **Exploratory Data Analysis**

**Univariate Analysis**

In [ ]:
fig = plt.figure(figsize= (15,9))
for i, col in enumerate(categorical_columns):
  ax=fig.add_subplot(2, 2, i+1)
  sns.countplot(x=Insurance[col], ax=ax)

fig.tight_layout()  
plt.show()

: 

In [ ]:
#Plot distribution of numerical columns
fig = plt.figure(figsize= (20,25))
for i, col in enumerate(numerical_columns):
    
    ax=fig.add_subplot( 5, 2, i+1)
    
    sns.distplot(Insurance[col])

fig.tight_layout()  
plt.show()

: 

**Bivariate Analysis**

In [ ]:
fig = px.scatter(Insurance,
                 x='children',
                 y='expenses',
                 width=800,
                 height=500,
                 title='Childern vs Expenses')
fig.update_traces(marker_size=5)
fig.show()

: 

In [ ]:
fig = px.scatter(Insurance,
                 x='age',
                 y='expenses',
                 width=800,
                 height=500,
                 title='Age vs Expenses')
fig.update_traces(marker_size=5)
fig.show()

: 

The general trend seems to be that medical expenses with age, as we might expect. However, there is significant veriation at every age, and it's clear that age alone cannot be used to accurately determine medical expenses.

In [ ]:
fig = px.scatter(Insurance,
                 x='children',
                 y='expenses',
                 color='smoker',
                 opacity=0.8,
                 hover_data=['sex'],
                 width=800,
                 height=500,
                 title='Childern vs Expenses')
fig.update_traces(marker_size=5)
fig.show()

: 

In [ ]:
fig = px.scatter(Insurance,
                 x='age',
                 y='expenses',
                 color='smoker',
                 opacity=0.8,
                 hover_data=['sex'],
                 width=800,
                 height=500,
                 title='Age vs Expenses')
fig.update_traces(marker_size=5)
fig.show()

: 

We can see three 'cluster' of points, each of which seems to form a line with an increasing slope:
* The first an dthe largest cluster consists primary of presumably 'healthy non-smokers' who have relatively low medical expenses compared to others.
* The second cluster contains a mix of smokers and non-smokers. it's possible that these are actually two distinct but overlapping clusters 'non-smokers with medical issue' and 'smokers without major medical issues'.
* The final cluster consists exclusively of smokers, persumably smokers with major medical issues that are possibly related to or worsened by smoking.

In [ ]:
fig = px.scatter(Insurance,
                 x='bmi',
                 y='expenses',
                 color='smoker',
                 opacity=0.8,
                 hover_data=['sex'],
                 width=800,
                 height=500,
                 title='Childern vs Expenses')
fig.update_traces(marker_size=5)
fig.show()

: 

It apperes that for non-smokers, an increase inBMi doesn't seem to be related to an increases in medical expenses. however, medical expenses seem to be significantly higher for smokers with b=BMI greater than 30.

**Now lets visulaize the correlation of categorical features to premium**

In [ ]:
var = 'sex'
data = pd.concat((Insurance[var], Insurance['expenses']),axis=1)

plt.subplots(figsize=(7,6))
fig = sns.boxplot(x=var, y='expenses', data=data)

: 

**Observation:**
From the box plot, expenses of female is less than male.

In [ ]:
var = 'smoker'
data = pd.concat((Insurance[var], Insurance['expenses']),axis=1)

plt.subplots(figsize=(7,6))
fig = sns.boxplot(x=var, y='expenses', data=data)

: 

**Observation:**
From the box plot, the non-smoker people insurance expenses is less tha somkers.

In [ ]:
var = 'region'
data = pd.concat((Insurance[var], Insurance['expenses']),axis=1)

plt.subplots(figsize=(7,6))
fig = sns.boxplot(x=var, y='expenses', data=data)

: 

**Observation:**
There is not much diff in median of all regions premium.

**Multivariate Analysis**

In [ ]:
sns.heatmap(Insurance.corr(),annot=True)

: 

**Observation:**
From the heatmap, expenses is 30% correlated to age and 20% to bmi.

# **Distribution**

In [ ]:
sns.displot(Insurance['expenses'])
plt.show()

: 

For most customers, the annual medical expenses are under 10k. only a small fraction of customer have higher medical expenses.

In [ ]:
print('Skewness: %f',Insurance['expenses'].skew())

: 

The rule of thumb seems to be: if the skewness is between -0.5 and 0.5, the data are fairly symmetrical. If the skewness is between -1 and -0.5 or 0.5 and 1, the data are moderatly skewed. If the skewness is less than -1 or greater than 1, the data are heighly skewed.

From this it is clear that expenses data is highly skewed.

we can say that, expenses data is:
* Deviated from the normal distribution
* Has positive skewness
* And shows peakedness

so, we need to normalize the data using Normalizing techniques(scaling, log-scaling, z-score)

In [ ]:
print('Kurtosis: %f',Insurance['expenses'].kurt())

: 

Positive kurtosis indicate a 'heavy-tailed'ndistribution and negative kurtosis indicate a 'light tailed' distribution.

From this, our data is heavly tailed.

* **Similary lets check how the other features are distributed**

In [ ]:
sns.displot(Insurance['bmi'])
plt.show()

: 

In [ ]:
print('Skewness: %f',Insurance['bmi'].skew())
print('Kurtosis: %f',Insurance['bmi'].kurt())

: 

This data looks fairly symmetrical and negatie kurtosis indicates that  the data is lightly tailed.

In [ ]:
sns.displot(Insurance['age'])
plt.show()

: 

In [ ]:
print('Skewness: %f',Insurance['age'].skew())
print('Kurtosis: %f',Insurance['age'].kurt())

: 

## **Feature Engineering**

Performing feature engineering to understand the different features in the dataset, and their relationship with the target variable.

It helps us understand the importance of every feature in the dataset.

Featurewiz identifies the most important features, and reduces the datset to have top 4 important features.

We have PCA as well to determain the most important features and performace dimensionality reduction.




#### **FeatureWiz**

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
for i, col in enumerate(categorical_columns):
  Insurance[col]=encoder.fit_transform(Insurance[col])

: 

In [ ]:
#!pip install featurewiz

: 

In [ ]:
from featurewiz import featurewiz

: 

In [ ]:
Insurance.head()

: 

In [ ]:
target = 'expenses'
feature = featurewiz(Insurance, target, corr_limit=0.70, verbose=2)

: 

### **Normalization**

In [ ]:
Insurance['expenses']=np.log10(Insurance['expenses'])

: 

In [ ]:
sns.displot(Insurance['expenses'])

: 

In [ ]:
X = Insurance.drop(['expenses'],axis=1)
y=Insurance['expenses']

: 

standard Scalar


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor 
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score
import pickle

: 

In [ ]:
#!pip install lazypredict

: 

In [ ]:
# Creatign new variables for numerical attributes/columns
numeric_features= numerical_columns
# Making pipeline for scaling down numeric features
numeric_transformer=Pipeline(steps=[('scaler', StandardScaler())]) 

# Creatign new variables for categorical attributes/columns
categoric_features = categorical_columns
# Making pipeline for Encoding categorical features
categorical_transformer=OneHotEncoder(handle_unknown='ignore')

# Creating new variable for these numerical and categorical features pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num',numeric_transformer, numeric_features),
        ('cat',categorical_transformer,categoric_features)
    ]
)


: 

In [ ]:
# Splitting the dataset fro training and testing the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

: 

In [ ]:
# Let us check shape of the traing and test set
print(X_train.shape,y_test.shape)

: 

## **Model**

In [ ]:
logger.info('Model Building')

: 

**Defining Baseline Models**
* In this section we will define all of the best possible basine models and train using the training data

In [ ]:
# Defining the scores list
model_scores = []

# Defining a list of useful regressors
models = {'Linear Regression': LinearRegression(),
          'Random Forest Regression': RandomForestRegressor(),
          'Decision Tree Regression': DecisionTreeRegressor(),
          'Gradient Boosting Regression': GradientBoostingRegressor(),
          'KNN': KNeighborsRegressor(),
          'XGBoost': XGBRegressor(random_state=42)}

: 

In [ ]:
for name, model in models.items():
    # Fit the model on train data
    model.fit(X_train, y_train)

    # Make predictions using train data
    y_train_pred = model.predict(X_train)

    # Make predictions using test data
    y_pred = model.predict(X_test)
    print(f'Model Name: {name}')
    print(f'RMSE Train: {np.sqrt(mean_squared_error(y_train, y_train_pred))}')
    print(f'RMSE Test: {np.sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'R2 Score: {r2_score(y_train, y_train_pred)}')
    print(f'R2 Score: {r2_score(y_test, y_pred)}')

    model_scores.append((name,
                       np.sqrt(mean_squared_error(y_train, y_train_pred)),
                       np.sqrt(mean_squared_error(y_test, y_pred))))

    print('*'*50)

   

: 

### **Model Performace Evaluation**


Formula
<center><img src = "https://www.gstatic.com/education/formulas2/472522532/en/root_mean_square_deviation.svg"></center>

RMSE	=	root-mean-square deviation

i	=	variable 

n	=	number of non-missing data points

x(i)	=	actual observations time series

xhat(i)	=	estimated time series

In [ ]:
models = pd.DataFrame(data=model_scores, columns=['Model', 'Train RMSE', 'Test RMSE'])
models.sort_values(by='Test RMSE', ascending=True)

: 

From the above observation we can say that the performace (RMSE) od Gradient Boosting Regressor is good as compared to other models.

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.05, 0.01],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Define the model
gb = GradientBoostingRegressor(random_state=42)

# Define the grid search with cross-validation
grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the best score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best RMSE score: ", np.sqrt(-grid_search.best_score_))

# Save the best model
joblib.dump(grid_search.best_estimator_, 'best_model.joblib')

: 